In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

def flatten_image(image):
    return np.array(image).flatten()

def flatten_images(images):
    return np.array([flatten_image(image) for image in images])

In [2]:
def read_image(class_path, image_file):
    image_path = os.path.join(class_path, image_file)
    return cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

class_labels_list = []
images_list = []

data_directory = './dataset/'

#missing O
directories = ['@', "&", "#", "$", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']

for class_label, class_name in enumerate(directories):
    class_path = os.path.join(data_directory, class_name)
    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpg'):
            image = read_image(class_path, image_file)
            images_list.append(image)
            class_labels_list.append(class_label)

data_x = images_list
data_y = class_labels_list

X_train, X_test, y_train, y_test = train_test_split(data_x,data_y, 
                                                    test_size=0.25, 
                                                    random_state=13)


In [4]:
pca = PCA(n_components=50).fit(flatten_images(X_train))
X_train_pca = pca.transform(flatten_images(X_train))
X_test_pca = pca.transform(flatten_images(X_test))


In [5]:

random_knn_classifier = KNeighborsClassifier(n_neighbors=5)

random_knn_classifier.fit(X_train_pca, y_train)

random_knn_predictions = random_knn_classifier.predict(X_test_pca)

random_knn_accuracy = accuracy_score(y_test, random_knn_predictions)

print("Random KNN PCA Classifier:")
print(f"Accuracy: {random_knn_accuracy}")

Random KNN PCA Classifier:
Accuracy: 0.881352187361909


In [6]:

def apply_sobel(image):
    gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
    gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)

    gradient = [gx, gy]
    
    return gradient

data_Xtrain_sobel = [apply_sobel(image) for image in X_train]
data_Xtest_sobel = [apply_sobel(image) for image in X_test]

In [7]:

data_Xtest_sobel = flatten_images(data_Xtest_sobel)
data_Xtrain_sobel = flatten_images(data_Xtrain_sobel)

random_knn_classifier = KNeighborsClassifier(n_neighbors=5)

random_knn_classifier.fit(data_Xtrain_sobel, y_train)

random_knn_predictions = random_knn_classifier.predict(data_Xtest_sobel)

random_knn_accuracy = accuracy_score(y_test, random_knn_predictions)

print("Random KNN Direction Classifier:")
print(f"Accuracy: {random_knn_accuracy}")

Random KNN Direction Classifier:
Accuracy: 0.8331860362350861


In [8]:
X_train_combined = np.concatenate((X_train_pca, data_Xtrain_sobel), axis=1)
X_test_combined = np.concatenate((X_test_pca, data_Xtest_sobel), axis=1)


random_knn_classifier = KNeighborsClassifier(n_neighbors=5)

random_knn_classifier.fit(X_train_combined, y_train)

random_knn_predictions = random_knn_classifier.predict(X_test_combined)

random_knn_accuracy = accuracy_score(y_test, random_knn_predictions)

print("Random KNN Combined Classifier:")
print(f"Accuracy: {random_knn_accuracy}")

Random KNN Combined Classifier:
Accuracy: 0.8331860362350861
